<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-09 10:13:10
-------------------
qualified stocks: 94
with latest results: 87
still star stocks: 47
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  6.31 L
Current:  1.20 C
-------------------
Today PnL: -1.62 L (-1.33%)
Current PnL: -25.04 L (-17.75%)
CY Booked + Current PnL: -22.20 L (-15.74%)
-------------------
Total profit:  2.40 L
Total loss:  -27.44 L
-------------------
Total Booked + Current PnL: 4.78 L (4.19%)
Total Booked PnL: 29.81 L (26.14%)
Curr Year Booked PnL: 2.83 L (2.35%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.05 C
Est FTT PnL: 84.23 L (69.92%)
-------------------
Est LTT:  2.66 C
Est LTT PnL: 1.45 C (120.63%)
Deployed:  1.14 C
Current:  1.20 C
CAGR/XIRR %: 3.96%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,MRF,114777.60,150504.12,18.39,X-MC,95.74,136200.0,21422.0,14301.0,-1.84,18.66,10.50,31.13,203.0,1.50,1.19,32.67,AR,ATH,TYRES
57,PIDILITIND,2504.06,3576.00,-13.58,X-LC,57.45,90192.0,15070.0,17091.0,1.65,20.06,18.95,42.81,66.0,0.88,0.79,13.11,X40,BTT,CHEMICALS
70,SBILIFE,1496.49,1924.99,-2.79,H-LC,91.49,144421.0,18716.0,17273.0,-0.70,14.89,11.96,28.63,116.0,1.08,1.27,28.66,AR,ATH,INSURANCE
71,SBIN,760.30,863.00,-19.48,M-LC,63.83,203520.0,2040.0,25175.0,-0.13,1.01,12.37,13.51,106.0,0.08,1.78,11.50,XY25,NTT,BANKS
35,ICICIGI,1852.76,2268.71,-20.98,H-LC,31.91,95213.0,-4836.0,27298.0,-1.60,-4.83,28.67,22.45,93.0,-0.18,0.83,15.52,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UNIONBANK,123.87,163.0,-14.24,M-LC,55.32,135462.0,-5378.0,49864.0,3.08,-3.82,36.81,31.59,102.0,-0.11,1.19,17.67,XY24,NTT,BANKS
81,TATAMOTORS,798.82,1065.0,-53.87,M-LC,93.62,143582.0,-20176.0,74749.0,2.57,-12.32,52.06,33.32,1.0,-0.27,1.26,20.81,XY24,NTT,AUTO
1,AARTIIND,487.04,919.0,21.36,M-SC,100.00,76645.0,-4691.0,76829.0,2.32,-5.77,100.24,88.69,114.0,-0.06,0.67,29.25,XR,NTT,CHEMICALS
57,PIDILITIND,2504.06,3576.0,-13.58,X-LC,57.45,90192.0,15070.0,17091.0,1.65,20.06,18.95,42.81,66.0,0.88,0.79,13.11,X40,BTT,CHEMICALS
45,LALPATHLAB,2798.50,3545.0,-33.16,H-SC,69.15,128777.0,46.0,34293.0,0.87,0.04,26.63,26.68,37.0,0.00,1.13,26.46,X40N,NTT,HEALTHCARE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,SURYODAY,189.78,240.0,27.19,H-SC,45.74,89712.0,-60214.0,99885.0,-7.59,-40.16,111.34,26.46,115.0,-0.60,0.79,14.93,XR,NTT,BANKS
24,DLF,684.82,1031.7,-32.01,H-LC,54.26,93862.0,-8861.0,60888.0,-4.47,-8.63,64.87,50.65,4.0,-0.15,0.82,2.91,AR,ATH,REALTY
8,ASIANTILES,75.41,137.0,4611.11,M-SC,41.49,51676.0,-42134.0,118751.0,-4.00,-44.91,229.80,81.67,145.0,-0.35,0.45,0.07,XR,NTT,CERAMICS
54,NHPC,82.08,115.0,173.68,M-MC,6.38,132595.0,-8418.0,64972.0,-3.77,-5.97,49.00,40.11,211.0,-0.13,1.16,8.40,XY24,NTT,POWER
23,DEN,52.79,75.0,73.58,L-SC,51.06,49040.0,-35424.0,70961.0,-3.68,-41.94,144.70,42.07,136.0,-0.50,0.43,4.15,AR,NTT,ENTERTAINMENT


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TATAMOTORS,798.82,1065.00,-53.87,M-LC,93.62,143582.0,-20176.0,74749.0,2.57,-12.32,52.06,33.32,1.0,-0.27,1.26,20.81,XY24,NTT,AUTO
79,TANLA,991.56,1963.11,-59.62,H-SC,59.57,102831.0,-116304.0,331013.0,-2.65,-53.07,321.90,97.98,2.0,-0.35,0.90,11.38,AR,ATH,IT
2,ABB,5319.37,8953.12,-41.33,H-LC,75.53,133062.0,78.0,90762.0,0.66,0.06,68.21,68.31,3.0,0.00,1.17,8.50,AR,ATH,ELECTRICAL
24,DLF,684.82,1031.70,-32.01,H-LC,54.26,93862.0,-8861.0,60888.0,-4.47,-8.63,64.87,50.65,4.0,-0.15,0.82,2.91,AR,ATH,REALTY
10,AWL,361.39,485.00,-70.76,H-MC,8.51,128647.0,-57469.0,121134.0,-1.77,-30.88,94.16,34.20,6.0,-0.47,1.13,5.96,XY24,NTT,FMCG


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,PIDILITIND,2504.06,3576.0,-13.58,X-LC,57.45,90192.0,15070.0,17091.0,1.65,20.06,18.95,42.81,66.0,0.88,0.79,13.11,X40,BTT,CHEMICALS


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,POLYCAB,5005.92,7541.35,12.69,H-MC,87.23,159964.0,19798.0,51188.0,-0.90,14.12,32.00,50.65,74.0,0.39,1.40,22.08,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,1.78,M-LC,90.43,179881.0,28675.0,36984.0,-1.51,18.96,20.56,43.43,45.0,0.78,1.58,24.69,XY24,NTT,MISC
71,SBIN,760.30,863.00,-19.48,M-LC,63.83,203520.0,2040.0,25175.0,-0.13,1.01,12.37,13.51,106.0,0.08,1.78,11.50,XY25,NTT,BANKS
45,LALPATHLAB,2798.50,3545.00,-33.16,H-SC,69.15,128777.0,46.0,34293.0,0.87,0.04,26.63,26.68,37.0,0.00,1.13,26.46,X40N,NTT,HEALTHCARE
92,WHIRLPOOL,1112.81,2270.00,-38.45,M-SC,88.30,109998.0,9845.0,94301.0,-1.04,9.83,85.73,103.99,8.0,0.10,0.96,32.60,X40,NTT,DURABLES


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
93,WIPRO,243.46,420.00,-13.34,M-LC,47.87,148502.0,-2443.0,111896.0,-0.85,-1.62,75.35,72.51,77.0,-0.02,1.30,11.81,XR,NTT,IT
12,BANKINDIA,113.49,190.00,-29.61,M-MC,50.00,123439.0,-5713.0,92777.0,0.57,-4.42,75.16,67.42,99.0,-0.06,1.08,20.06,XR,NTT,BANKS
1,AARTIIND,487.04,919.00,21.36,M-SC,100.00,76645.0,-4691.0,76829.0,2.32,-5.77,100.24,88.69,114.0,-0.06,0.67,29.25,XR,NTT,CHEMICALS
91,VOLTAS,1272.00,1929.20,-22.21,H-MC,11.70,132374.0,-7546.0,79835.0,-1.61,-5.39,60.31,51.67,9.0,-0.09,1.16,0.00,AR,ATH,AC
18,CAMS,3643.00,5250.99,-12.99,H-SC,15.96,96337.0,-5667.0,50693.0,-3.60,-5.56,52.62,44.14,26.0,-0.11,0.84,12.72,XR,ATH,MISC


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,UJJIVANSFB,52.77,60.00,-43.55,M-SC,65.96,109269.0,-33210.0,52733.0,-2.34,-23.31,48.26,13.70,139.0,-0.63,0.96,28.97,X40N,NTT,BANKS
52,MRF,114777.60,150504.12,18.39,X-MC,95.74,136200.0,21422.0,14301.0,-1.84,18.66,10.50,31.13,203.0,1.50,1.19,32.67,AR,ATH,TYRES
70,SBILIFE,1496.49,1924.99,-2.79,H-LC,91.49,144421.0,18716.0,17273.0,-0.70,14.89,11.96,28.63,116.0,1.08,1.27,28.66,AR,ATH,INSURANCE
61,RBLBANK,230.61,388.00,26.78,M-SC,89.36,196992.0,-37308.0,197209.0,-1.26,-15.92,100.11,68.25,207.0,-0.19,1.73,29.73,XY24,NTT,BANKS
69,SBIETFPB,241.24,400.00,NaN,NaN,80.85,206230.0,23611.0,96578.0,-1.07,12.93,46.83,65.81,NaN,0.24,1.81,15.16,XY24,BTT,BANKS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,AARTIIND,487.04,919.00,21.36,M-SC,100.00,76645.0,-4691.0,76829.0,2.32,-5.77,100.24,88.69,114.0,-0.06,0.67,29.25,XR,NTT,CHEMICALS
76,SONACOMS,578.05,1006.00,-33.18,M-MC,98.94,86818.0,-14341.0,89232.0,-2.36,-14.18,102.78,74.03,7.0,-0.16,0.76,22.01,AR,BTT,AUTO
90,VIPIND,488.80,718.00,-746.89,H-SC,97.87,61620.0,-33696.0,78393.0,-0.02,-35.35,127.22,46.89,39.0,-0.43,0.54,21.54,X40N,NTT,MISC
38,INDUSINDBK,1354.50,1800.00,-37.01,H-MC,96.81,46694.0,-30512.0,55907.0,-0.92,-39.52,119.73,32.89,46.0,-0.55,0.41,28.59,XR,NTT,BANKS
52,MRF,114777.60,150504.12,18.39,X-MC,95.74,136200.0,21422.0,14301.0,-1.84,18.66,10.50,31.13,203.0,1.50,1.19,32.67,AR,ATH,TYRES


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.99
1,25,41.95
2,50,69.93


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    35.80
X40N    14.03
X40     13.76
AR      12.15
XY25    11.51
XR      10.40
X200     1.21
SR       1.15
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.22
X-LC    19.07
M-SC    16.33
M-LC     8.25
M-MC     7.59
X-MC     7.02
H-MC     6.40
H-LC     5.62
X-SC     2.64
L-SC     2.25
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
BANKS,11.34,-16.53,73.68
FMCG,11.22,-5.40,37.67
IT,9.19,-29.80,97.75
FINANCE,7.40,-30.83,89.88
MISC,6.45,-40.16,90.88
PAINTS,5.84,-23.37,59.88
INSURANCE,5.58,-9.86,43.01
ELECTRICAL,4.24,-15.14,73.82
REFINERIES,3.83,6.80,10.38


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3342614.0,28
XR,1357122.0,15
X40N,1297958.0,17
AR,1109393.0,13
X40,800947.0,12
XY25,222050.0,6
SR,213745.0,2
X200,79652.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2427049.0,21
M-SC,2057234.0,20
X-LC,806016.0,12
M-MC,725190.0,7
H-MC,391472.0,6
X-SC,375917.0,4
M-LC,374844.0,6
X-MC,353290.0,6
L-SC,341728.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      995954.0      6
M-SC       XY24      841984.0      6
H-SC       X40N      605343.0      7
M-MC       XY24      513866.0      4
M-SC       XR        457832.0      5
X-LC       X40       407978.0      5
M-SC       X40N      405112.0      5
H-SC       AR        394349.0      2
           XR        336281.0      4
M-LC       XY24      237773.0      4
X-LC       AR        196582.0      3
L-SC       XR        170047.0      2
M-SC       X40       169125.0      2
H-LC       AR        168923.0      3
H-MC       XY24      168445.0      2
X-MC       XY24      155108.0      2
X-SC       XR        132382.0      1
           SR        125774.0      1
X-LC       XY25      121804.0      3
X-MC       X40N      118554.0      2
X-SC       X40N      117761.0      2
M-LC       XR        111896.0      1
L-SC       XY24      100720.0      1
M-SC       AR         95210.0      1
H-SC       X40        95122.0      2
M-MC       XR         92777.0      1
           AR         89232.0      1
M-SC       SR         87971.0      1
H-MC       AR         79835.0      1
X-LC       X200       79652.0      1
L-SC       AR         70961.0      1
X-MC       X40        65327.0      1
H-MC       XR         55907.0      1
           X40N       51188.0      1
H-LC       XY25       45756.0      1
H-MC       X40        36097.0      1
M-MC       XY25       29315.0      1
H-LC       X40        27298.0      1
M-LC       XY25       25175.0      1
X-MC       AR         14301.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
